In [1]:
import sys
sys.path.append(r"D:\\timesformer")

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode
from pathlib import Path
import numpy as np
import random
from decord import VideoReader
from tqdm import tqdm
from timesformer.models.vit import TimeSformer
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

torch.cuda.empty_cache()

# ----------------- 하이퍼파라미터 및 경로 ----------------------------
TRAIN_ROOT = Path(r"D:\golfDataset\dataset\train")
TEST_ROOT  = Path(r"D:\golfDataset\dataset\test")
PRETRAIN_PTH = Path(r"D:\timesformer\pretrained\TimeSformer_divST_8x32_224_K600.pyth")
NUM_FRAMES = 8
IMG_SIZE = 224
CLIPS_PER_VIDEO = 1

BATCH_SIZE = 4
EPOCHS = 10
SEED = 42

EPOCHS_DIR = Path("epochs")
EPOCHS_DIR.mkdir(exist_ok=True)
CHECKPOINT_PATH = Path("checkpoint.pth")

# ----------------- 재현성 ----------------------------
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ----------------- 데이터 증강 transform 정의 ----------------------------
train_transform = T.Compose([
    T.ToPILImage(),
    T.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    T.ToTensor(),
    T.Normalize([0.45]*3, [0.225]*3)
])

test_transform = T.Compose([
    T.ToPILImage(),
    T.Resize(256, interpolation=InterpolationMode.BICUBIC),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize([0.45]*3, [0.225]*3)
])

def preprocess_tensor(img_tensor, train=True):
    if train:
        img = train_transform(img_tensor)
    else:
        img = test_transform(img_tensor)
    return img

def uniform_sample(L, N):
    if L >= N:
        return np.linspace(0, L-1, N).astype(int)
    return np.pad(np.arange(L), (0, N-L), mode='edge')

def load_clip(path: Path, train=True):
    vr = VideoReader(str(path))
    L = len(vr)
    seg_edges = np.linspace(0, L, 2, dtype=int)
    clips = []
    for s0, s1 in zip(seg_edges[:-1], seg_edges[1:]):
        idx = uniform_sample(s1 - s0, NUM_FRAMES) + s0
        arr = vr.get_batch(idx).asnumpy().astype(np.uint8)
        clip = torch.from_numpy(arr).permute(0, 3, 1, 2).float() / 255.0
        clip = torch.stack([preprocess_tensor(f, train=train) for f in clip])
        clips.append(clip.permute(1, 0, 2, 3))
    return clips

class TimeSformerWithDropout(nn.Module):
    def __init__(self, base_model, dropout_p):
        super().__init__()
        self.base = base_model
        self.dropout = nn.Dropout(dropout_p)
        self.head = base_model.model.head

    def forward(self, x):
        feats = self.base.model.forward_features(x)
        out = self.dropout(feats)
        return self.head(out)

class GolfSwingDataset(Dataset):
    def __init__(self, root: Path, balance, train=True):
        self.samples = []
        self.train = train
        true_samples = []
        false_samples = []
        for label, sub in [(1, "balanced_true"), (0, "false")]:
            for p in (root/sub/"crop_video").glob("*.mp4"):
                if label == 1:
                    true_samples.append((p, 1))
                else:
                    false_samples.append((p, 0))
        if balance:
            n_true = len(true_samples)
            n_false = len(false_samples)
            if n_false > 0 and n_true > n_false:
                factor = n_true // n_false
                remainder = n_true % n_false
                false_samples = false_samples * factor + false_samples[:remainder]
        self.samples = true_samples + false_samples
        random.shuffle(self.samples)
        print(f"✅ {len(self.samples)} samples found in {root} (balanced={balance})")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        clips = load_clip(path, train=self.train)
        return torch.stack(clips), torch.tensor(label)

def get_trainable_params(model, train_block):
    params = []
    total_blocks = 12
    for name, param in model.base.model.named_parameters():
        if any(f"blocks.{i}." in name for i in range(total_blocks - train_block, total_blocks)):
            param.requires_grad = True
            params.append(param)
        else:
            param.requires_grad = False
    for param in model.base.model.head.parameters():
        param.requires_grad = True
        params.append(param)
    return params

# ----------------- GA 기반 하이퍼파라미터 튜닝 ----------------------------
LR_RANGE = (1e-6, 1e-4)
WD_RANGE = (0.001, 0.01)
DO_RANGE = (0.3, 0.6)
TFRATIO_RANGE = (1.0, 2.0)
TB_SPACE = [2]  # TRAIN_BLOCK 2로 고정

POP_SIZE = 6
N_GEN = 4
ELITE = 2
MUT_PROB = 0.3

def random_individual():
    return {
        "LR": float(10 ** random.uniform(np.log10(LR_RANGE[0]), np.log10(LR_RANGE[1]))),
        "WEIGHT_DECAY": float(random.uniform(WD_RANGE[0], WD_RANGE[1])),
        "DROPOUT": float(random.uniform(DO_RANGE[0], DO_RANGE[1])),
        "TF_RATIO": [float(random.uniform(TFRATIO_RANGE[0], TFRATIO_RANGE[1])), 1.0],
        "TRAIN_BLOCK": random.choice(TB_SPACE)
    }

def mutate(ind):
    if random.random() < MUT_PROB:
        ind["LR"] = float(10 ** random.uniform(np.log10(LR_RANGE[0]), np.log10(LR_RANGE[1])))
    if random.random() < MUT_PROB:
        ind["WEIGHT_DECAY"] = float(random.uniform(WD_RANGE[0], WD_RANGE[1]))
    if random.random() < MUT_PROB:
        ind["DROPOUT"] = float(random.uniform(DO_RANGE[0], DO_RANGE[1]))
    if random.random() < MUT_PROB:
        ind["TF_RATIO"] = [float(random.uniform(TFRATIO_RANGE[0], TFRATIO_RANGE[1])), 1.0]
    if random.random() < MUT_PROB:
        ind["TRAIN_BLOCK"] = random.choice(TB_SPACE)
    return ind

def crossover(parent1, parent2):
    child = {}
    for k in parent1.keys():
        child[k] = random.choice([parent1[k], parent2[k]])
    return child

results = []
population = [random_individual() for _ in range(POP_SIZE)]

for gen in range(N_GEN):
    print(f"\n=== Generation {gen+1} ===")
    scores = []
    for i, indiv in enumerate(population):
        print(f"\n[GA] Individual {i+1}: {indiv}")

        # 데이터셋
        train_dataset = GolfSwingDataset(TRAIN_ROOT, balance=True, train=True)
        test_dataset  = GolfSwingDataset(TEST_ROOT, balance=False, train=False)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
        test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

        # 모델
        base_model = TimeSformer(img_size=IMG_SIZE, num_frames=NUM_FRAMES,
                                 num_classes=2, attention_type='divided_space_time',
                                 pretrained_model=str(PRETRAIN_PTH)).to(DEVICE)
        model = TimeSformerWithDropout(base_model, dropout_p=indiv["DROPOUT"]).to(DEVICE)
        optimizer = optim.AdamW(get_trainable_params(model, indiv["TRAIN_BLOCK"]), lr=indiv["LR"], weight_decay=indiv["WEIGHT_DECAY"])
        criterion = nn.CrossEntropyLoss(weight=torch.tensor(indiv["TF_RATIO"]).to(DEVICE))

        PATIENCE = 3
        BEST_TEST_ACC = 0
        PATIENCE_CNT = 0
        OVERFIT_GAP = 0.35

        for epoch in range(EPOCHS):
            model.train()
            total = correct = total_loss = 0
            for clips, label in tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]", ncols=70, leave=False):
                B = clips.shape[0]
                clips = clips.view(-1, *clips.shape[2:]).to(DEVICE)
                labs = label.repeat(CLIPS_PER_VIDEO).to(DEVICE)
                outs = model(clips)
                loss = criterion(outs, labs)
                optimizer.zero_grad(); loss.backward(); optimizer.step()
                total_loss += loss.item()
                total += labs.size(0)
                correct += (outs.argmax(1) == labs).sum().item()
            train_acc = correct / total
            avg_loss = total_loss / total
            print(f"  train acc: {train_acc:.3%}, loss: {avg_loss:.4f}")

            # --- 각 epoch마다 test 평가 ---
            model.eval()
            test_total = test_correct = 0
            with torch.no_grad():
                for clips, label in tqdm(test_loader, desc=f"Epoch {epoch+1} [Test]", ncols=70, leave=False):
                    B = clips.shape[0]
                    vids = clips.view(-1, *clips.shape[2:]).to(DEVICE)
                    labs = label.repeat(CLIPS_PER_VIDEO).to(DEVICE)
                    probs = model(vids).softmax(1)
                    pred = probs.argmax(1)
                    test_total += pred.size(0)
                    test_correct += (pred.cpu() == labs.cpu()).sum().item()
            test_acc = test_correct / test_total if test_total > 0 else 0
            print(f"  test  acc: {test_acc:.3%}")

            # --- Early Stopping 조건 ---
            if test_acc > BEST_TEST_ACC:
                BEST_TEST_ACC = test_acc
                PATIENCE_CNT = 0
            else:
                PATIENCE_CNT += 1

            if train_acc - test_acc > OVERFIT_GAP:
                print(f"⚠️ Early stopping (overfitting): train_acc={train_acc:.2%}, test_acc={test_acc:.2%}")
                break

            if PATIENCE_CNT >= PATIENCE:
                print(f"⏹️ Early stopping (no test acc improvement for {PATIENCE} epochs)")
                break

        # 평가 (마지막 epoch 기준)
        model.eval()
        test_total = test_correct = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for clips, label in tqdm(test_loader, desc="Test", ncols=70, leave=False):
                B = clips.shape[0]
                vids = clips.view(-1, *clips.shape[2:]).to(DEVICE)
                labs = label.repeat(CLIPS_PER_VIDEO).to(DEVICE)
                probs = model(vids).softmax(1)
                pred = probs.argmax(1)
                test_total += pred.size(0)
                test_correct += (pred.cpu() == labs.cpu()).sum().item()
                all_preds.extend(pred.cpu().numpy())
                all_labels.extend(labs.cpu().numpy())
        test_acc = test_correct / test_total if test_total > 0 else 0
        print(f"  test  acc: {test_acc:.3%}")

        # 아래처럼 BEST_TEST_ACC를 사용!
        indiv["test_acc"] = BEST_TEST_ACC
        scores.append((BEST_TEST_ACC, indiv))
        results.append(indiv.copy())

    # 상위 elite 개체 선발
    scores.sort(reverse=True, key=lambda x: x[0])
    elites = [indiv for _, indiv in scores[:ELITE]]

    # 다음 세대 생성 (교배+변이)
    next_population = elites.copy()
    while len(next_population) < POP_SIZE:
        p1, p2 = random.sample(elites, 2)
        child = crossover(p1, p2)
        child = mutate(child)
        next_population.append(child)
    population = next_population

# 결과 정렬 및 출력
results = sorted(results, key=lambda x: x["test_acc"], reverse=True)
print("\n=== 하이퍼파라미터 튜닝 결과 (상위 5개) ===")
for r in results[:5]:
    print(r)


=== Generation 1 ===

[GA] Individual 1: {'LR': 1.9004375238737116e-05, 'WEIGHT_DECAY': 0.0012250967970040025, 'DROPOUT': 0.38250879551073574, 'TF_RATIO': [1.2232107381488229, 1.0], 'TRAIN_BLOCK': 2}
✅ 2404 samples found in D:\golfDataset\dataset\train (balanced=True)
✅ 295 samples found in D:\golfDataset\dataset\test (balanced=False)


  train acc: 76.331%, loss: 0.1165


  test  acc: 43.051%


  train acc: 91.639%, loss: 0.0508


  test  acc: 42.373%
⚠️ Early stopping (overfitting): train_acc=91.64%, test_acc=42.37%


...

  train acc: 66.306%, loss: 0.1506


  test  acc: 51.186%


  train acc: 72.088%, loss: 0.1341


  test  acc: 50.847%
⏹️ Early stopping (no test acc improvement for 3 epochs)


  test  acc: 50.847%

=== 하이퍼파라미터 튜닝 결과 (상위 5개) ===
{'LR': 1.6011077254941482e-06, 'WEIGHT_DECAY': 0.004419345705736037, 'DROPOUT': 0.4076938141453885, 'TF_RATIO': [1.343955722478971, 1.0], 'TRAIN_BLOCK': 2, 'test_acc': 0.5423728813559322}
{'LR': 1.6011077254941482e-06, 'WEIGHT_DECAY': 0.005650822060912167, 'DROPOUT': 0.4918223242920275, 'TF_RATIO': [1.006498759678061, 1.0], 'TRAIN_BLOCK': 2, 'test_acc': 0.5254237288135594}
{'LR': 2.7598230915285763e-06, 'WEIGHT_DECAY': 0.006303391154883179, 'DROPOUT': 0.542829137003348, 'TF_RATIO': [1.006498759678061, 1.0], 'TRAIN_BLOCK': 2, 'test_acc': 0.5220338983050847}
{'LR': 1.6011077254941482e-06, 'WEIGHT_DECAY': 0.007428308726019948, 'DROPOUT': 0.513007569140643, 'TF_RATIO': [1.9009566196929555, 1.0], 'TRAIN_BLOCK': 2, 'test_acc': 0.5220338983050847}
{'LR': 1.6011077254941482e-06, 'WEIGHT_DECAY': 0.0021273159474264674, 'DROPOUT': 0.4464679848486083, 'TF_RATIO': [1.614289001017703, 1.0], 'TRAIN_BLOCK': 2, 'test_acc': 0.5186440677966102}
